In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import threading
import concurrent.futures

In [2]:
"1230".isdigit()

True

In [3]:
def find_elements_with_class(html_content, *class_name):
    try:
      
        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all elements with the specified class
        elements_with_class = soup.find_all(class_=list(class_name))
        #print(list(class_name))

        return elements_with_class
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [4]:
def find_first_elements_with_class(html_content, *class_name):
    try:
      
        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all elements with the specified class
        elements_with_class = soup.find(class_=list(class_name))
        #print(list(class_name))

        return elements_with_class
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [5]:
def transformTag(x):

    x = str(x)
    
    #print(x)
    pos = x.find('>')

    pos2 = x.find('</')

    return x[pos+1:pos2]

In [6]:
everything = dict()

keys = ['Pret','Oferit de','Are VIN (Serie sasiu)','Marca','Model','Versiune','Generatie','Anul fabricației','Km','Combustibil','Putere','Capacitate cilindrica','Transmisie','Norma de poluare','Cutie de viteze','Tip Caroserie','Numar de portiere','VIN (serie sasiu)','Culoare','Numar locuri','Optiuni culoare','Se emite factura','Eligibil pentru finantare','Garantie de la producator pana la','sau in limita a','Tara de origine','Data primei inmatriculari','Inmatriculat','Primul proprietar (de nou)','Fara accident in istoric','Carte de service','Stare','Consum Extraurban','Consum Urban',]

for key in keys:
    everything[key] = []

In [7]:
import requests
from tqdm import tqdm
def read_html_from_url(url):
    try:
        response = requests.get(url)

        if response.status_code == 200:
            html_content = response.text
            return html_content
        else:
            print("Failed to retrieve the HTML file. Status code:", response.status_code)
            return None
            
    except requests.RequestException as e:
        print("Error fetching the HTML file:", e)
        return None


In [8]:
def process_url(link):
    link = link.strip()
    #print(link)
    #break
    url_to_read = link
    html_content = read_html_from_url(url_to_read)

    interest = find_elements_with_class(html_content,"e18eslyg4 ooa-12b2ph5","e16lfxpc0 ooa-1pe3502 er34gjf0","e16lfxpc1 ooa-1ftbcn2","offer-price__number eqdspoq4 ooa-o7wv9s er34gjf0","offer-price__currency eqdspoq5 ooa-pk2ls er34gjf0")
    #print(interest)
    
    
    
    keys,values = [],[]
    for i in range(0,len(interest),2):
        key = transformTag(interest[i])
        #print(interest[i+1])
        value = transformTag(interest[i+1])

        auxKey = "".join(key.split())
        #print(auxKey)
        if auxKey.isdigit():
            value = f"{key} {value}"
            key = 'Pret'
        keys.append(key)
        values.append(value)
        
    #print(keys)
    # #print(html_content)
    # keys = find_elements_with_class(html_content,"e18eslyg4 ooa-12b2ph5")

    # keys = [transformTag(x) for x in keys]
    # #print(keys)


    # values = find_elements_with_class(html_content,"e16lfxpc0 ooa-1pe3502 er34gjf0","e16lfxpc1 ooa-1ftbcn2")

    # values = [transformTag(x) for x in values]

    # prices = find_first_elements_with_class(html_content,"offer-price__number eqdspoq4 ooa-o7wv9s er34gjf0")
    
    # #prices = find_elements_with_class(html_content,"offer-price__number eqdspoq4 ooa-o7wv9s er34gjf0","offer-price__currency eqdspoq5 ooa-pk2ls er34gjf0")

    # prices = transformTag(prices)
    # #print(prices)
    # currencies = find_first_elements_with_class(html_content,"offer-price__currency eqdspoq5 ooa-pk2ls er34gjf0")
    # currencies = transformTag(currencies)

    
    # #print(prices)
    # keys.append('Pret')
    # values.append(f"{prices} {currencies}")
    #print(values)

    data = list(zip(keys,values))

    return keys,values

In [9]:



fin = open("links.txt","r")

thr = []

with concurrent.futures.ThreadPoolExecutor() as executor:

    cnt = 0
    
    for link in tqdm(fin.readlines()):
        cnt+=1
        thr.append(executor.submit(process_url, link))
        # if cnt==10:
        #     break

    cnt = 0
    for thread in tqdm(thr):
        keys,values = thread.result()
        
        for x,y in everything.items():
            if x not in keys:
                everything[x].append('-')
            else:
                idx = keys.index(x)
                #print(idx)
                everything[x].append(values[idx])
        cnt+=1 
        # if cnt==10:
        #     break
        #everything['Pret'] = prices
        #print(*key,sep='\n')
        
        # if html_content:
        
        #     with open(f"ceva.txt","w") as fout:
        #         # Displaying the HTML content fetched from the URL
    
        #         html_content = html_content
    
        #         print(html_content)
        # else:
    
        #     print(f"ERROR {page}")

100%|█████████████████████████████████████| 37601/37601 [46:09<00:00, 13.58it/s]


In [10]:
#print(everything)

In [11]:
df = pd.DataFrame(everything)
df.head(500)

,Pret,Oferit de,Are VIN (Serie sasiu),Marca,Model,Versiune,Generatie,Anul fabricației,Km,Combustibil,...,sau in limita a,Tara de origine,Data primei inmatriculari,Inmatriculat,Primul proprietar (de nou),Fara accident in istoric,Carte de service,Stare,Consum Extraurban,Consum Urban
0,16 900 EUR,Firma,Da,Ford,Mondeo,2.0 TDCi Powershift ST Line High,-,2016,94 640 km,Diesel,...,-,Romania,04 aprilie 2017,Da,Da,Da,Da,Second hand,"4,50 l/100km","5,40 l/100km"
1,58 905 EUR,Firma,Da,Volvo,XC 60,B5 MHEV AT8 AWD Inscription,II [2017 - Prezent],2023,5 000 km,Diesel,...,150 000 km,Romania,martie 2023,Da,Da,Da,Da,Second hand,-,-
2,12 300 EUR,Firma,-,Volkswagen,Tiguan,2.0 TSI 4Motion DSG,-,2013,254 000 km,Benzina,...,-,Germania,-,-,Da,Da,Da,Second hand,-,8 l/100km
3,13 299 EUR,Firma,-,Mazda,CX-3,CD105 Attraction,-,2018,137 289 km,Diesel,...,5 000 km,Belgia,09 ianuarie 2018,-,-,Da,-,Second hand,"3,80 l/100km","4,40 l/100km"
4,13 500 EUR,Firma,-,Audi,Q3,-,-,2014,140 000 km,Diesel,...,-,Germania,-,-,Da,Da,Da,Second hand,-,6 l/100km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,8 990 EUR,Privat,-,Volvo,V40,D4 VEA R-Design,II [2012 - Prezent],2014,229 553 km,Diesel,...,-,Olanda,25 septembrie 2014,-,Da,-,Da,Second hand,-,-
496,36 990 EUR,Privat,Da,BMW,Seria 3,320i AT,G20/G21 [2018 - Prezent],2021,40 286 km,Benzina,...,-,Germania,27 iulie 2021,-,-,Da,Da,Second hand,-,"7,70 l/100km"
497,12 969 EUR,Firma,-,Nissan,Qashqai,1.2 DIG-T Start/Stop Tekna,-,2015,53 600 km,Benzina,...,-,-,-,-,-,Da,Da,Second hand,-,"6,80 l/100km"
498,7 500 EUR,Firma,Da,Ford,Fiesta,1.5 TDCi Titanium,VIII [2017 - Prezent],2019,142 123 km,Diesel,...,-,-,-,-,-,-,-,Second hand,"3,10 l/100km","3,80 l/100km"


In [12]:
df.to_csv('./autovit.csv')

In [33]:
df = pd.read_csv("./autovit.csv")

/tmp/ipykernel_11043/961072984.py:1: DtypeWarning: Columns (8,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./autovit.csv")


In [34]:
print(df.columns)

Index(['Unnamed: 0', 'Pret', 'Oferit de', 'Are VIN (Serie sasiu)', 'Marca',
       'Model', 'Versiune', 'Generatie', 'Anul fabricației', 'Km',
       'Combustibil', 'Putere', 'Capacitate cilindrica', 'Transmisie',
       'Norma de poluare', 'Cutie de viteze', 'Tip Caroserie',
       'Numar de portiere', 'VIN (serie sasiu)', 'Culoare', 'Numar locuri',
       'Optiuni culoare', 'Se emite factura', 'Eligibil pentru finantare',
       'Garantie de la producator pana la', 'sau in limita a',
       'Tara de origine', 'Data primei inmatriculari', 'Inmatriculat',
       'Primul proprietar (de nou)', 'Fara accident in istoric',
       'Carte de service', 'Stare', 'Consum Extraurban', 'Consum Urban'],
      dtype='object')


In [35]:
df2 = df.drop(columns=['Data primei inmatriculari','Generatie','VIN (serie sasiu)','Carte de service','Consum Extraurban','Consum Urban','Unnamed: 0'])
df2.head(50)

,Pret,Oferit de,Are VIN (Serie sasiu),Marca,Model,Versiune,Anul fabricației,Km,Combustibil,Putere,...,Optiuni culoare,Se emite factura,Eligibil pentru finantare,Garantie de la producator pana la,sau in limita a,Tara de origine,Inmatriculat,Primul proprietar (de nou),Fara accident in istoric,Stare
0,16 900 EUR,Firma,Da,Ford,Mondeo,2.0 TDCi Powershift ST Line High,2016,94 640 km,Diesel,150 CP,...,Metalizata,Da,Da,-,-,Romania,Da,Da,Da,Second hand
1,58 905 EUR,Firma,Da,Volvo,XC 60,B5 MHEV AT8 AWD Inscription,2023,5 000 km,Diesel,235 CP,...,Metalizata,Da,Da,1 Februarie 2027,150 000 km,Romania,Da,Da,Da,Second hand
2,12 300 EUR,Firma,-,Volkswagen,Tiguan,2.0 TSI 4Motion DSG,2013,254 000 km,Benzina,180 CP,...,-,Da,Da,-,-,Germania,-,Da,Da,Second hand
3,13 299 EUR,Firma,-,Mazda,CX-3,CD105 Attraction,2018,137 289 km,Diesel,105 CP,...,-,Da,Da,-,5 000 km,Belgia,-,-,Da,Second hand
4,13 500 EUR,Firma,-,Audi,Q3,-,2014,140 000 km,Diesel,140 CP,...,-,-,Da,-,-,Germania,-,Da,Da,Second hand
5,17 969 EUR,Firma,-,Opel,Corsa,1.2 Turbo Start/Stop Aut. Edition,2022,1 km,Benzina,100 CP,...,Mat,Da,Da,-,-,-,-,Da,Da,Nou
6,24 514 EUR,Firma,-,Opel,Grandland,-,2022,1 km,Benzina,130 CP,...,Metalizata,Da,Da,-,-,-,-,Da,Da,Nou
7,15 950 EUR,Firma,Da,Land Rover,Range Rover Evoque,-,2012,239 000 km,Diesel,190 CP,...,Metalizata,Da,Da,-,-,-,-,Da,Da,Second hand
8,60 500 EUR,Firma,Da,Mercedes-Benz,C,-,2023,9 000 km,Diesel,288 CP,...,Metalizata,Da,Da,-,-,-,Da,-,Da,Second hand
9,18 490 EUR,Firma,-,BMW,Seria 3,-,2018,73 066 km,Benzina,136 CP,...,Metalizata,Da,Da,-,-,Romania,Da,Da,Da,Second hand


In [36]:
print(set(list(df['Stare'])))

{'Second hand', 'Nou', '-'}


In [37]:
df2['Are VIN (Serie sasiu)'] = df2['Are VIN (Serie sasiu)'].replace('-', 'Nu')
df2['Inmatriculat'] = df2['Inmatriculat'].replace('-', 'Nu')
df2['Primul proprietar (de nou)'] = df2['Primul proprietar (de nou)'].replace('-', 'Nu')
df2['Fara accident in istoric'] = df2['Fara accident in istoric'].replace('-', 'Nu')
df2['Stare'] = df2['Stare'].replace('-', 'Second hand')
df2['Garantie de la producator pana la'] = df2['Garantie de la producator pana la'].replace('-', 'Nu')
df2['sau in limita a'] = df2['sau in limita a'].replace('-', 'Nu')
df2['Eligibil pentru finantare'] = df2['Eligibil pentru finantare'].replace('-', 'Nu')
df2['Se emite factura'] = df2['Se emite factura'].replace('-', 'Nu')

In [38]:
df2.replace('-', None, inplace=True)
df2.head(5)

,Pret,Oferit de,Are VIN (Serie sasiu),Marca,Model,Versiune,Anul fabricației,Km,Combustibil,Putere,...,Optiuni culoare,Se emite factura,Eligibil pentru finantare,Garantie de la producator pana la,sau in limita a,Tara de origine,Inmatriculat,Primul proprietar (de nou),Fara accident in istoric,Stare
0,16 900 EUR,Firma,Da,Ford,Mondeo,2.0 TDCi Powershift ST Line High,2016,94 640 km,Diesel,150 CP,...,Metalizata,Da,Da,Nu,Nu,Romania,Da,Da,Da,Second hand
1,58 905 EUR,Firma,Da,Volvo,XC 60,B5 MHEV AT8 AWD Inscription,2023,5 000 km,Diesel,235 CP,...,Metalizata,Da,Da,1 Februarie 2027,150 000 km,Romania,Da,Da,Da,Second hand
2,12 300 EUR,Firma,Nu,Volkswagen,Tiguan,2.0 TSI 4Motion DSG,2013,254 000 km,Benzina,180 CP,...,None,Da,Da,Nu,Nu,Germania,Nu,Da,Da,Second hand
3,13 299 EUR,Firma,Nu,Mazda,CX-3,CD105 Attraction,2018,137 289 km,Diesel,105 CP,...,None,Da,Da,Nu,5 000 km,Belgia,Nu,Nu,Da,Second hand
4,13 500 EUR,Firma,Nu,Audi,Q3,None,2014,140 000 km,Diesel,140 CP,...,None,Nu,Da,Nu,Nu,Germania,Nu,Da,Da,Second hand


In [39]:
df2 = df2.dropna()

In [40]:
df2.head(10000)

,Pret,Oferit de,Are VIN (Serie sasiu),Marca,Model,Versiune,Anul fabricației,Km,Combustibil,Putere,...,Optiuni culoare,Se emite factura,Eligibil pentru finantare,Garantie de la producator pana la,sau in limita a,Tara de origine,Inmatriculat,Primul proprietar (de nou),Fara accident in istoric,Stare
0,16 900 EUR,Firma,Da,Ford,Mondeo,2.0 TDCi Powershift ST Line High,2016,94 640 km,Diesel,150 CP,...,Metalizata,Da,Da,Nu,Nu,Romania,Da,Da,Da,Second hand
1,58 905 EUR,Firma,Da,Volvo,XC 60,B5 MHEV AT8 AWD Inscription,2023,5 000 km,Diesel,235 CP,...,Metalizata,Da,Da,1 Februarie 2027,150 000 km,Romania,Da,Da,Da,Second hand
18,13 490 EUR,Firma,Nu,Renault,Espace,Energy dCi 160 EDC Intens,2016,194 000 km,Diesel,160 CP,...,Perlat,Da,Da,Nu,Nu,Austria,Nu,Da,Da,Second hand
27,14 190 EUR,Firma,Nu,Renault,Megane,Blue dCi EDC Limited,2019,151 725 km,Diesel,116 CP,...,Metalizata,Da,Da,Nu,30 000 km,Romania,Da,Da,Da,Second hand
30,17 500 EUR,Firma,Da,BMW,Seria 3,320i AT,2017,208 600 km,Benzina,184 CP,...,Metalizata,Da,Da,Nu,Nu,Germania,Nu,Nu,Da,Second hand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37587,31 900 EUR,Firma,Nu,Volvo,XC 60,B5 MHEV AT8 AWD Inscription,2020,172 000 km,Diesel,235 CP,...,Metalizata,Nu,Nu,Nu,Nu,Germania,Da,Da,Nu,Second hand
37589,9 490 EUR,Firma,Da,Volkswagen,Jetta,1.4 TSI DSG Highline,2012,238 000 km,Benzina,122 CP,...,Perlat,Da,Da,Nu,10 000 km,Romania,Da,Da,Da,Second hand
37593,19 500 EUR,Privat,Da,Renault,Megane,ENERGY dCi 165 EDC BOSE EDITION,2018,90 000 km,Diesel,163 CP,...,Metalizata,Nu,Nu,Nu,Nu,Franta,Da,Da,Da,Second hand
37597,9 799 EUR,Privat,Nu,Volkswagen,Golf Sportsvan,2.0 TDI (BlueMotion Technology) Comfortline,2015,245 000 km,Diesel,150 CP,...,Perlat,Nu,Nu,Nu,Nu,Germania,Nu,Nu,Nu,Second hand


In [41]:
df2.to_csv('./filtered.csv')